### Zadanie 1
Wykorzystaj funkcję finfo z biblioteki numpy aby określić precyzję i zakres różnych typów danych reprezentujących liczby zmienno-przecinkowe. Wyniki badania przedstaw w postaci zestawienia tabelrycznego.

In [1]:
import numpy as np
from tabulate import tabulate

table = tabulate([['float64', np.finfo(np.float64).precision,
                   f'{np.finfo(np.float64).min} to {np.finfo(np.float64).max}'],
                  ['float32', np.finfo(np.float32).precision,
                   f'{np.finfo(np.float32).min} to {np.finfo(np.float32).max}'],
                  ['float16', np.finfo(np.float16).precision,
                   f'{np.finfo(np.float16).min} to {np.finfo(np.float16).max}']],
                 headers=['Type', 'Precision', 'Range'])
print(table)

Type       Precision  Range
-------  -----------  ---------------------------------------------------
float64           15  -1.7976931348623157e+308 to 1.7976931348623157e+308
float32            6  -3.4028234663852886e+38 to 3.4028234663852886e+38
float16            3  -65504.0 to 65504.0


### Zadanie 2 
Napisz funkcje, która dla dowolnej liczby zmiennoprzecinkowej w precyzji 64-bitowej (w standardzie IEEE 754) przedstawi w wersji dziesiętnej i binarnej składniki liczby: znak, wykładnik i mantysa. Można wykorzystać między innymi bibliotekę bitstring. Zbadaj jak reprezentowaną są poszczególne składniki tego formatu: zakres, sposób kodowania liczby etc.

In [4]:
!pip install bitstring
!pip install tabulate
import bitstring as bs


def show_bin_and_dec(number):
    float64 = bs.BitArray(float=number, length=64)
    exponent = float64.bin[1:12]
    mantissa = float64.bin[12:]

    if float64[0] == 1:
        sign = '-'
    else:
        sign = '+'

    exponent_dec = 0
    pow = 10
    for i in exponent[0:]:
        if i == '1':
            exponent_dec += 2 ** pow
        pow -= 1

    pow = -1
    mantissa_dec = 0
    for i in mantissa[0:]:
        if i == '1':
            mantissa_dec += 2 ** pow
        pow -= 1

    print('Floating point numbers (64-Bit):', '\n')
    print('number (dec) = sign (1 + mantissa) x 2 ** (exponent - 1023)')
    print(number, '(dec) =', sign, f'(1 + {mantissa_dec}) x 2 ** ({exponent_dec} - 1023) = '
                                   f'{sign}{1 + mantissa_dec} x 2 ** {exponent_dec - 1023}', '\n')
    print("Sign Exponent Mantissa")
    print(float64.bin[0], exponent, mantissa)
    print(sign, exponent_dec, mantissa_dec)


show_bin_and_dec(1.2)
show_bin_and_dec(1.75)
show_bin_and_dec(1.25)


Floating point numbers (64-Bit): 

number (dec) = sign (1 + mantissa) x 2 ** (exponent - 1023)
1.25 (dec) = + (1 + 0.25) x 2 ** (1023 - 1023) = +1.25 x 2 ** 0 

Sign Exponent Mantissa
0 01111111111 0100000000000000000000000000000000000000000000000000
+ 1023 0.25
Floating point numbers (64-Bit): 

number (dec) = sign (1 + mantissa) x 2 ** (exponent - 1023)
1.5 (dec) = + (1 + 0.5) x 2 ** (1023 - 1023) = +1.5 x 2 ** 0 

Sign Exponent Mantissa
0 01111111111 1000000000000000000000000000000000000000000000000000
+ 1023 0.5
Floating point numbers (64-Bit): 

number (dec) = sign (1 + mantissa) x 2 ** (exponent - 1023)
-0.6875 (dec) = - (1 + 0.375) x 2 ** (1022 - 1023) = -1.375 x 2 ** -1 

Sign Exponent Mantissa
1 01111111110 0110000000000000000000000000000000000000000000000000
- 1022 0.375


Liczby zmiennoprzecinkowe w precyzji 64-bitowej (wg standadru IEEE 754) są przechowywane w postaci 3 części:
1. Pierwszy bit od (od lewej) oznacza znak. Dla 0 mamy liczbę dodatnią a 1 oznacza -.
2. Następnie jest 11 bitów cechy. Za jej pomocą można "przesuwać" przecinek w liczbie. Jest to możliwe w obie storny (pozwala na reprezentację zarówno bardzo dużych jak i bardzo małych liczb - przez możliwość zwiększenia miejsca na część całkowitą lub ułamkową, zależnie od potrzeb). Pierwszy bit cechy oznacza, podobnie jak pierwszy bit całej liczby, znak. Liczba 'cecha - 1023' jest używana jako wykładnik potęgi 2 właśnie w celu przesuwania przecinka (stąd nazwa "floating point - reprezentacja zmiennoprzecikowa")
3. Na końcu zanjdują się 52 bity mantysy, kodującej właściwą liczbę. W celu normalizacji zawsze (w domyśle) dodawane jest do niech 1.

### Zadanie 3 
Wykorzystaj funkcję nextafter() z biblioteki numpy (lub inny sposób) i sprawdź rozdzielczość typu float (jakie mają wartości najbliższe sąsiadujące liczby float). Zaprezentuj wynik korzystając z funkcji z Zadania 2. 

In [6]:
import numpy as np
# from zad2 import show_bin_and_dec


def next_float(x):
    print("Next (closest) value to {} is {}".format(x, np.nextafter(x, 1)))
    print(show_bin_and_dec(x))
    print(show_bin_and_dec(np.nextafter(x, 1)))


next_float(1.2)
next_float(1.75)
next_float(1.25)


Next (closest) value to 1.2 is 1.1999999999999997
Floating point numbers (64-Bit): 

number (dec) = sign (1 + mantissa) x 2 ** (exponent - 1023)
1.2 (dec) = + (1 + 0.19999999999999996) x 2 ** (1023 - 1023) = +1.2 x 2 ** 0 

Sign Exponent Mantissa
0 01111111111 0011001100110011001100110011001100110011001100110011
+ 1023 0.19999999999999996
None
Floating point numbers (64-Bit): 

number (dec) = sign (1 + mantissa) x 2 ** (exponent - 1023)
1.1999999999999997 (dec) = + (1 + 0.19999999999999973) x 2 ** (1023 - 1023) = +1.1999999999999997 x 2 ** 0 

Sign Exponent Mantissa
0 01111111111 0011001100110011001100110011001100110011001100110010
+ 1023 0.19999999999999973
None
Next (closest) value to 1.75 is 1.7499999999999998
Floating point numbers (64-Bit): 

number (dec) = sign (1 + mantissa) x 2 ** (exponent - 1023)
1.75 (dec) = + (1 + 0.75) x 2 ** (1023 - 1023) = +1.75 x 2 ** 0 

Sign Exponent Mantissa
0 01111111111 1100000000000000000000000000000000000000000000000000
+ 1023 0.75
None
Floating 

### Zadanie 4
Porównać reprezentację bitową liczby 1/3 dla typów float o różnej precyzji: float16, float32, float64. Sprawdź co się dzieje gdy zmienna o mniejszej precyzji konwertowana jest do wyższej. 

In [7]:
import numpy as np


hex2bin = dict('{:x} {:04b}'.format(x, x).split() for x in range(16))


def float_dec2bin(d):
    hx = float(d).hex()
    p = hx.index('p')
    bn = ''.join(hex2bin.get(char, char) for char in hx[2:p])
    return (bn.strip('0') + hx[p:p + 2]
            + bin(int(hx[p + 2:]))[2:]).split("p")[0]


print("1/3 float to binary")
print("float64:", float_dec2bin(np.float64(1/10)))
print("float32:", float_dec2bin(np.float32(1/3)))
print("float16:", float_dec2bin(np.float16(1/3)))

print("float16 -> float32:", float_dec2bin(np.float32(np.float16(1/3))))
print("float32 -> float64:", float_dec2bin(np.float64(np.float32(1/3))))
print("float16 -> float64:", float_dec2bin(np.float16(np.float32(1/3))))


1/3 float to binary
float64: 1.100110011001100110011001100110011001100110011001101
float32: 1.01010101010101010101011
float16: 1.0101010101
float16 -> float32: 1.0101010101
float32 -> float64: 1.01010101010101010101011
float16 -> float64: 1.0101010101


Konwersja liczby z reprezentacji o mniejszej precyzji do większej nie ma większego sensu, gdyż liczba zapisana w mniejszej precyzji zawiera mniej informacji (jest mniej dokładna ze względu). Stąd też, kiedy konwertujemy ją do wyższej precyzji, nie zyskujemy nic ponieważ nasze dodatkowe (nowe) bity będą po prostu zerami. Z kolei konwersja w drugą stronę, tj. do mniejszych precyzji spowoduje utatę dokładności zapisanej liczby (czyli utratę części informacji o niej) - niejako "odetneimy" część bitów kodujących naszą liczbę.

### Zadanie 5
Sprawdź czym jest postać znormalizowana liczby typu float. Poniższy program generuje ciąg liczb. Wyświetl ich postać binarną i sprawdź czy są w postaci znormalizowanej (napisz stosowny kod, który to sprawdza). 

In [25]:
a=np.float32(1.1)
for i in range(0, 149):
    a=a/np.float32(2.0)

### Zadanie 6
Wyjaśnij dlaczego poniższy program nie działa prawidłowo. Sprawdź co się stanie gdy użyjemy typów o mniejszej precyzji. Zaproponuj inny sposób sumowania, który będzie dawał dokładne wyniki. 

In [26]:
N = 100000000
C = 0.09531258654533566;

sum = 0.0 
for i in range(0, N):
    sum += C
    
print(sum)
print(N*C)

9531258.666974738
9531258.654533565


Powyższy błąd wynika z faktu, że liczby zmiennoprzecinkowe obarczone są pewnym błędem, spowodowanym skończoną liczbą bitów używanych do reprezentacji (potencjalnie nieskończonej liczby nieskończonych) wartości (czyli o skończonej precyzji). Dlatego też, sumując odpowiednio dużo razy wyżej podaną liczbę, za każdym razem zwiększamy nasz błąd. Wpewnym momencie wartość powyższej sumy może zacząć również przekraczać zakres wartości typu używanego do jej przechowywania. Poniżej prezentuję zestawienie dla sumowania "wprost" w 3 różnych stopniach precyzji oraz sumowanie z użyciem Algorytmu Sumacyjnego Kahana. Algorytm ten został stowrzony właśnie w celu zminimalizowania błędów powodowanych przez sumowanie liczb zmiennoprzecinkowych o skończonej precyzji.

In [9]:
import numpy as np
from tabulate import tabulate


def kahan_algorithm(x, n):
    sum = 0.0
    eps = 0.0
    for i in range(0, n):
        x_without_eps = x - eps
        tmp_sum = sum + x_without_eps
        eps = (tmp_sum - sum) - x_without_eps
        sum = tmp_sum
    return sum


def add_n_times(x, n, sum):
    for i in range(0, n):
        sum += x
    return sum


table = [["float16", add_n_times(np.float16(0.09531258654533566), 100000000, np.float16(0))],
        ["float32", add_n_times(np.float32(0.09531258654533566), 100000000, np.float32(0))],
        ["float64", add_n_times(np.float64(0.09531258654533566), 100000000, np.float64(0))],
        ["kahan algorithm", kahan_algorithm(0.09531258654533566, 100000000)]]

print(tabulate(table, headers=['Type', 'Value'], floatfmt=".10f"))

Type                          Value
---------------  ------------------
float16              256.0000000000
float32          2097152.0000000000
float64          9531258.6669747382
kahan algorithm  9531258.6545335650
